In [ ]:
# =========================================================
# LEMBAR KERJA PERTEMUAN 6 - RANDOM FOREST KLASIFIKASI
# =========================================================

# Langkah 1 — Muat Data & Split (Pilihan A)
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

# Import modul sklearn
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_recall_curve

print("==========================================")
print("LANGKAH 1: Muat Data & Split 70/15/15")
print("==========================================")

# >>> FILE SAVE/LOAD <<< : Muat data yang sudah diproses
try:
    df = pd.read_csv(r"C:\.venv\Machine-learning\dataset.csv")
except FileNotFoundError:
    print("ERROR: Pastikan file 'processed_kelulusan.csv' ada di direktori ini.")
    exit()

X = df.drop("Lulus", axis=1)
y = df["Lulus"]

# Split 1: Train (70%) dan Temporary (30%) - Menggunakan Stratified
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

# Split 2: Temporary (30%) dibagi 50/50 menjadi Val (15%) dan Test (15%)
# Stratify dihapus untuk menghindari ValueError karena data kecil
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, random_state=42 
)

print(f"Train Shape: {X_train.shape}, Val Shape: {X_val.shape}, Test Shape: {X_test.shape}")
print(f"Proporsi Lulus di Train: {y_train.mean():.2f}")


# =========================================================
# LANGKAH 2 — Pipeline & Baseline Random Forest
# =========================================================
print("\n==========================================")
print("LANGKAH 2: Pipeline & Baseline Random Forest")
print("==========================================")

num_cols = X_train.select_dtypes(include="number").columns

pre = ColumnTransformer([
    ("num", Pipeline([("imp", SimpleImputer(strategy="median")),
                      ("sc", StandardScaler())]), num_cols),
], remainder="drop")

rf = RandomForestClassifier(
    n_estimators=300, max_features="sqrt",
    class_weight="balanced", random_state=42
)

pipe = Pipeline([("pre", pre), ("clf", rf)])
pipe.fit(X_train, y_train)

y_val_pred = pipe.predict(X_val)
print("Baseline RF — F1(val):", f1_score(y_val, y_val_pred, average="macro"))
print("Baseline Classification Report (Val):\n", classification_report(y_val, y_val_pred, digits=3))


# =========================================================
# LANGKAH 3 — Validasi Silang (Cross-Validation)
# =========================================================
print("\n==========================================")
print("LANGKAH 3: Validasi Silang (Cross-Validation)")
print("==========================================")

skf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
scores = cross_val_score(pipe, X_train, y_train, cv=skf, scoring="f1_macro", n_jobs=-1)
print(f"CV F1-macro (train): {scores.mean():.4f} ± {scores.std():.4f}")


# =========================================================
# LANGKAH 4 — Tuning Ringkas (GridSearch)
# =========================================================
print("\n==========================================")
print("LANGKAH 4: Tuning Ringkas (GridSearch)")
print("==========================================")

param = {
  "clf__max_depth": [None, 3, 5],
  "clf__min_samples_split": [2, 5]
}

gs = GridSearchCV(pipe, param_grid=param, cv=skf,
                  scoring="f1_macro", n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)

best_model = gs.best_estimator_
y_val_best = best_model.predict(X_val)

print("Best params:", gs.best_params_)
print("Best RF — F1(val):", f1_score(y_val, y_val_best, average="macro"))



print("\n==========================================")
print("LANGKAH 5: Evaluasi Akhir (Test Set)")
print("==========================================")

final_model = best_model  

y_test_pred = final_model.predict(X_test)
print("F1(test):", f1_score(y_test, y_test_pred, average="macro"))
print("Classification Report (Test):\n", classification_report(y_test, y_test_pred, digits=3))
print("Confusion Matrix (test):\n", confusion_matrix(y_test, y_test_pred))

if hasattr(final_model, "predict_proba"):
    y_test_proba = final_model.predict_proba(X_test)[:,1]
    
    try:
        roc_auc = roc_auc_score(y_test, y_test_proba)
        print("ROC-AUC(test):", roc_auc)
    except ValueError as e:
        print(f"ROC-AUC tidak dapat dihitung: {e}.") 

    fpr, tpr, _ = roc_curve(y_test, y_test_proba)
    plt.figure()
    plt.plot(fpr, tpr)
    plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC Curve (Test) - AUC: {roc_auc:.3f} (Jika Terhitung)")
    plt.show() # Tampilkan plot

    prec, rec, _ = precision_recall_curve(y_test, y_test_proba)
    plt.figure()
    plt.plot(rec, prec)
    plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title("PR Curve (Test)")
    plt.show() # Tampilkan plot



print("\n==========================================")
print("LANGKAH 6: Pentingnya Fitur (Feature Importance)")
print("==========================================")

try:
    importances = final_model.named_steps["clf"].feature_importances_
    fn = final_model.named_steps["pre"].get_feature_names_out()
    
    feature_names = [name.split('__')[1] for name in fn]

    top = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)
    
    print("Top feature importance:")
    for name, val in top:
        print(f"{name}: {val:.4f}")
except Exception as e:
    print("Feature importance tidak tersedia atau gagal:", e)



print("\n==========================================")
print("LANGKAH 7: Simpan Model")
print("==========================================")

# >>> FILE SAVE/LOAD <<<: Menyimpan model
joblib.dump(final_model, "rf_model.pkl")
print("Model disimpan sebagai rf_model.pkl")



print("\n==========================================")
print("LANGKAH 8: Cek Inference Lokal")
print("==========================================")


mdl = joblib.load("rf_model.pkl")


sample = pd.DataFrame([{
  "IPK": 3.4,
  "Jumlah_Absensi": 4,
  "Waktu_Belajar_Jam": 7,
  "Rasio_Absensi": 4/14, 
  "IPK_x_Study": 3.4*7
}])

prediksi = mdl.predict(sample)[0]
print(f"Input Sample: IPK 3.4, Absensi 4, Waktu Belajar 7")
print("Prediksi Kelulusan (1=Lulus, 0=Tidak Lulus):", int(prediksi))

LANGKAH 1: Muat Data & Split 70/15/15
ERROR: Pastikan file 'processed_kelulusan.csv' ada di direktori ini.


NameError: name 'df' is not defined